## **Imports**

In [129]:
from docplex.mp.model import Model
import numpy as np

## **1. From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)**

### **Define the ILP formulation of the BPP**

In [114]:
def ilp_model_of_bpp(sizes, bin_capacity):
    
    model = Model(name='BinPackingProblem')

    n = len(sizes)
    m = n  # Max bins needed is n (one bin per item)

    # Decision variables
    x = model.binary_var_matrix(n, m, name='x')  # item i is asing in item j
    y = model.binary_var_list(m, name='y')  # y_j: whether bin j is used

    # Objective: Minimize the number of bins used
    model.minimize(model.sum(y[j] for j in range(m)))

    # Constraints:

    # Constraint 1
    # Each item must be assigned to exactly one bin
    for i in range(n):
        model.add_constraint(model.sum(x[i, j] for j in range(m)) == 1)

    # Constrain 2
    # The sum of item sizes in each bin cannot exceed bin capacity
    for j in range(m):
        model.add_constraint(model.sum(sizes[i] * x[i, j] for i in range(n)) <= bin_capacity * y[j])

    return model

def ilp_to_qubo(sizes, bin_capacity, penalty_weight=1):
    n = len(sizes)
    m = n  # Max bins

    # Initialize QUBO matrix (m * n binary variables)
    Q = np.zeros((m * n, m * n))

    # Minimize the number of bins (linear term for y_j)
    for j in range(m):
        for i in range(n):
            Q[i*m + j, i*m + j] -= 1  # Objective to minimize number of bins

    # Penalize if item i is not assigned exactly once
    for i in range(n):
        for j1 in range(m):
            for j2 in range(j1 + 1, m):
                Q[i*m + j1, i*m + j2] += penalty_weight  # Add penalty for multiple assignments

    # Penalize bin capacity violations
    for j in range(m):
        for i1 in range(n):
            for i2 in range(i1, n):
                Q[j * n + i1, j * n + i2] += penalty_weight * sizes[i1] * sizes[i2] / bin_capacity

    return Q



sizes = [3, 2, 5]
bin_capacity = 6
Q_small = ilp_to_qubo(sizes, bin_capacity)
# print("QUBO Matrix for Small Instance:")
# print(Q_small)

model = create_bpp_ilp_model(sizes,bin_capacity)

model.prettyprint()

// This file has been generated by DOcplex
// model name is: BinPackingProblem
// var contrainer section
dvar bool x[3][3];
dvar bool y[3];

minimize
 y_0 + y_1 + y_2;
 
subject to {
 x_0_0 + x_0_1 + x_0_2 == 1;
 x_1_0 + x_1_1 + x_1_2 == 1;
 x_2_0 + x_2_1 + x_2_2 == 1;
 3 x_0_0 + 2 x_1_0 + 5 x_2_0 <= 6 y_0;
 3 x_0_1 + 2 x_1_1 + 5 x_2_1 <= 6 y_1;
 3 x_0_2 + 2 x_1_2 + 5 x_2_2 <= 6 y_2;

}


In [115]:
sizes = [3, 2, 5, 4, 7, 6]
bin_capacity = 10
Q_medium = ilp_to_qubo(sizes, bin_capacity)
print("QUBO Matrix for Medium Instance:")
print(Q_medium)


QUBO Matrix for Medium Instance:
[[-0.1  1.6  2.5 ...  0.   0.   0. ]
 [ 0.  -0.6  2.  ...  0.   0.   0. ]
 [ 0.   0.   1.5 ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  0.6  3.8  3.4]
 [ 0.   0.   0.  ...  0.   3.9  5.2]
 [ 0.   0.   0.  ...  0.   0.   2.6]]


In [121]:
sizes = [3, 2, 5]
bin_capacity = 6

model_1 = QuadraticProgram_BPP_integer_linear(sizes,bin_capacity)
model_2 = create_bpp_ilp_model(sizes,bin_capacity)
print(model_1.prettyprint())
print(model_2.prettyprint())

Problem name: 

Minimize
  y_1 + y_2 + y_3

Subject to
  Linear constraints (6)
    x_11 + x_12 + x_13 == 1  'object_1_assigned_constraint'
    x_21 + x_22 + x_23 == 1  'object_2_assigned_constraint'
    x_31 + x_32 + x_33 == 1  'object_3_assigned_constraint'
    3*x_11 + 2*x_21 + 5*x_31 - 6*y_1 <= 0  'container_1_capacity_constraint'
    3*x_12 + 2*x_22 + 5*x_32 - 6*y_2 <= 0  'container_2_capacity_constraint'
    3*x_13 + 2*x_23 + 5*x_33 - 6*y_3 <= 0  'container_3_capacity_constraint'

  Binary variables (12)
    y_1 y_2 y_3 x_11 x_12 x_13 x_21 x_22 x_23 x_31 x_32 x_33

// This file has been generated by DOcplex
// model name is: BinPackingProblem
// var contrainer section
dvar bool x[3][3];
dvar bool y[3];

minimize
 y_0 + y_1 + y_2;
 
subject to {
 x_0_0 + x_0_1 + x_0_2 == 1;
 x_1_0 + x_1_1 + x_1_2 == 1;
 x_2_0 + x_2_1 + x_2_2 == 1;
 3 x_0_0 + 2 x_1_0 + 5 x_2_0 <= 6 y_0;
 3 x_0_1 + 2 x_1_1 + 5 x_2_1 <= 6 y_1;
 3 x_0_2 + 2 x_1_2 + 5 x_2_2 <= 6 y_2;

}
None
